# Análise Exploratória de Dados
## Hackathon Participa DF - Categoria Acesso à Informação

Este notebook contém a análise exploratória dos dados de pedidos de acesso à informação.

**Objetivo**: Identificar pedidos que contenham dados pessoais (nome, CPF, RG, telefone, e-mail)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pathlib import Path
import sys

# Adicionar src ao path
sys.path.append('../src')

from preprocessamento import carregar_dados, preprocessar_dados
from modelo import DetectorPadroesRegex, DetectorNER

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

%matplotlib inline
%load_ext autoreload
%autoreload 2

print("Ambiente configurado com sucesso!")

## 1. Carregamento dos Dados

In [ ]:
# Carregar dados de treino
# NOTA: Ajuste o caminho conforme os dados fornecidos pela CGDF
caminho_dados = '../dados/treino/pedidos_treino.csv'

# Verificar se o arquivo existe
if Path(caminho_dados).exists():
    df = carregar_dados(caminho_dados)
    print(f"Dados carregados com sucesso!")
    print(f"Shape: {df.shape}")
else:
    print(f"AVISO: Arquivo não encontrado: {caminho_dados}")
    print("Criando dados de exemplo para demonstração...")
    
    # Dados de exemplo
    df = pd.DataFrame({
        'id_pedido': range(1, 11),
        'texto_pedido': [
            "Gostaria de informações sobre o orçamento de 2025",
            "Meu CPF é 123.456.789-00 e preciso de informações sobre meu processo",
            "Solicito dados sobre obras públicas no DF",
            "Entre em contato pelo email joao.silva@email.com",
            "Qual o status do processo administrativo?",
            "Meu telefone é (61) 98765-4321 para retorno",
            "Pedro Santos da Silva solicita informações",
            "Relatório de gastos públicos em 2024",
            "RG 1.234.567 SSP/DF necessário para análise",
            "Informações sobre licitações abertas"
        ],
        'contem_dados_pessoais': [False, True, False, True, False, True, True, False, True, False]
    })
    print("Dados de exemplo criados.")

# Mostrar primeiras linhas
df.head()

## 2. Visão Geral dos Dados

In [ ]:
# Informações básicas
print("Informações do Dataset:")
print(f"Total de pedidos: {len(df)}")
print(f"Colunas: {df.columns.tolist()}")
print(f"\nTipos de dados:")
print(df.dtypes)
print(f"\nValores nulos:")
print(df.isnull().sum())

In [ ]:
# Distribuição da variável alvo
if 'contem_dados_pessoais' in df.columns:
    print("Distribuição de Classes:")
    print(df['contem_dados_pessoais'].value_counts())
    print(f"\nProporção:")
    print(df['contem_dados_pessoais'].value_counts(normalize=True))
    
    # Plotar
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Contagem
    df['contem_dados_pessoais'].value_counts().plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'])
    axes[0].set_title('Distribuição de Classes', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Contém Dados Pessoais')
    axes[0].set_ylabel('Quantidade')
    axes[0].set_xticklabels(['Não', 'Sim'], rotation=0)
    
    # Proporção
    df['contem_dados_pessoais'].value_counts(normalize=True).plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_title('Proporção de Classes', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
else:
    print("Coluna 'contem_dados_pessoais' não encontrada (dados de teste).")

## 3. Análise dos Textos

In [ ]:
# Estatísticas dos textos
df['tamanho_texto'] = df['texto_pedido'].apply(len)
df['qtd_palavras'] = df['texto_pedido'].apply(lambda x: len(str(x).split()))
df['qtd_numeros'] = df['texto_pedido'].apply(lambda x: len(re.findall(r'\d', str(x))))

print("Estatísticas dos Textos:")
print(df[['tamanho_texto', 'qtd_palavras', 'qtd_numeros']].describe())

In [ ]:
# Distribuição do tamanho dos textos
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Tamanho do texto
axes[0, 0].hist(df['tamanho_texto'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribuição do Tamanho dos Textos (caracteres)', fontweight='bold')
axes[0, 0].set_xlabel('Tamanho')
axes[0, 0].set_ylabel('Frequência')

# Quantidade de palavras
axes[0, 1].hist(df['qtd_palavras'], bins=30, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Distribuição da Quantidade de Palavras', fontweight='bold')
axes[0, 1].set_xlabel('Quantidade de Palavras')
axes[0, 1].set_ylabel('Frequência')

# Quantidade de números
axes[1, 0].hist(df['qtd_numeros'], bins=30, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Distribuição da Quantidade de Números', fontweight='bold')
axes[1, 0].set_xlabel('Quantidade de Números')
axes[1, 0].set_ylabel('Frequência')

# Boxplot por classe (se disponível)
if 'contem_dados_pessoais' in df.columns:
    df.boxplot(column='qtd_numeros', by='contem_dados_pessoais', ax=axes[1, 1])
    axes[1, 1].set_title('Quantidade de Números por Classe', fontweight='bold')
    axes[1, 1].set_xlabel('Contém Dados Pessoais')
    axes[1, 1].set_ylabel('Quantidade de Números')
    plt.suptitle('')
else:
    axes[1, 1].text(0.5, 0.5, 'Rótulos não disponíveis', ha='center', va='center')
    axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 4. Detecção de Padrões com Regex

In [ ]:
# Inicializar detector de padrões
detector_regex = DetectorPadroesRegex()

# Detectar padrões em todos os textos
print("Detectando padrões com regex...")
deteccoes = df['texto_pedido'].apply(detector_regex.detectar_todos)

# Expandir dicionários em colunas
df_deteccoes = pd.DataFrame(deteccoes.tolist())
df_combined = pd.concat([df, df_deteccoes], axis=1)

# Estatísticas de detecção
print("\nPadrões Detectados:")
print(f"CPF: {df_deteccoes['cpf'].sum()} pedidos")
print(f"RG: {df_deteccoes['rg'].sum()} pedidos")
print(f"Telefone: {df_deteccoes['telefone'].sum()} pedidos")
print(f"E-mail: {df_deteccoes['email'].sum()} pedidos")
print(f"\nQualquer padrão: {df_deteccoes.any(axis=1).sum()} pedidos")

In [ ]:
# Visualizar detecções
fig, ax = plt.subplots(figsize=(10, 6))

tipos_dados = ['cpf', 'rg', 'telefone', 'email']
contagens = [df_deteccoes[tipo].sum() for tipo in tipos_dados]

bars = ax.bar(tipos_dados, contagens, color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12'])
ax.set_title('Quantidade de Detecções por Tipo de Dado Pessoal', fontsize=14, fontweight='bold')
ax.set_xlabel('Tipo de Dado')
ax.set_ylabel('Quantidade de Pedidos')

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Exemplos de Pedidos

In [ ]:
# Exemplos com dados pessoais detectados por regex
if df_deteccoes.any(axis=1).sum() > 0:
    print("Exemplos de pedidos COM dados pessoais (detectados por regex):\n")
    exemplos_com = df_combined[df_deteccoes.any(axis=1)].head(5)
    
    for idx, row in exemplos_com.iterrows():
        print(f"ID: {row['id_pedido']}")
        print(f"Texto: {row['texto_pedido'][:200]}...")
        detectados = [tipo for tipo in tipos_dados if row[tipo]]
        print(f"Detectados: {', '.join(detectados)}")
        print("-" * 80)
else:
    print("Nenhum padrão detectado nos dados de exemplo.")

In [ ]:
# Exemplos sem dados pessoais
if (~df_deteccoes.any(axis=1)).sum() > 0:
    print("\nExemplos de pedidos SEM dados pessoais (não detectados por regex):\n")
    exemplos_sem = df_combined[~df_deteccoes.any(axis=1)].head(5)
    
    for idx, row in exemplos_sem.iterrows():
        print(f"ID: {row['id_pedido']}")
        print(f"Texto: {row['texto_pedido'][:200]}...")
        print("-" * 80)
else:
    print("Todos os pedidos contêm algum padrão detectado.")

## 6. Análise de Performance (se rótulos disponíveis)

In [ ]:
if 'contem_dados_pessoais' in df.columns:
    # Comparar detecções com rótulos verdadeiros
    df_combined['detectado_regex'] = df_deteccoes.any(axis=1)
    
    from sklearn.metrics import classification_report, confusion_matrix
    
    y_true = df_combined['contem_dados_pessoais']
    y_pred = df_combined['detectado_regex']
    
    print("Performance da Detecção por Regex (baseline):\n")
    print(classification_report(y_true, y_pred, target_names=['Sem dados', 'Com dados']))
    
    # Matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Sem dados', 'Com dados'],
                yticklabels=['Sem dados', 'Com dados'])
    plt.title('Matriz de Confusão - Detecção por Regex', fontweight='bold')
    plt.ylabel('Valor Real')
    plt.xlabel('Valor Predito')
    plt.tight_layout()
    plt.show()
    
    # Análise de erros
    print("\nAnálise de Erros:")
    falsos_negativos = df_combined[(y_true == True) & (y_pred == False)]
    print(f"\nFalsos Negativos (não detectados): {len(falsos_negativos)}")
    if len(falsos_negativos) > 0:
        print("Exemplos:")
        for idx, row in falsos_negativos.head(3).iterrows():
            print(f"  - {row['texto_pedido'][:150]}...")
    
    falsos_positivos = df_combined[(y_true == False) & (y_pred == True)]
    print(f"\nFalsos Positivos (detectados incorretamente): {len(falsos_positivos)}")
    if len(falsos_positivos) > 0:
        print("Exemplos:")
        for idx, row in falsos_positivos.head(3).iterrows():
            print(f"  - {row['texto_pedido'][:150]}...")
else:
    print("Rótulos não disponíveis para análise de performance.")

## 7. Próximos Passos

Com base nesta análise exploratória, os próximos passos são:

1. **Melhorar Detecção de Regex**:
   - Ajustar padrões para reduzir falsos negativos
   - Adicionar validação mais rigorosa para reduzir falsos positivos

2. **Implementar NER**:
   - Usar spaCy para detectar nomes próprios
   - Filtrar falsos positivos (nomes de lugares, organizações)

3. **Treinar Modelo ML**:
   - TF-IDF + Random Forest/Logistic Regression
   - Validação cruzada para avaliar generalização

4. **Ensemble**:
   - Combinar regex, NER e ML para maximizar F1-Score
   - Ajustar thresholds para priorizar recall

5. **Avaliação Final**:
   - Testar no conjunto de controle da CGDF
   - Otimizar para minimizar falsos negativos

## 8. Salvar Análise

In [ ]:
# Salvar dados com detecções
caminho_saida = '../resultados/dados_com_deteccoes.csv'
Path(caminho_saida).parent.mkdir(parents=True, exist_ok=True)
df_combined.to_csv(caminho_saida, index=False)
print(f"Dados com detecções salvos em: {caminho_saida}")